Script to extract GloFAS reforecast data at station locations stored in an s3 bucket. Metadata file is used to identify which station points to extract (use Lisflood x and y coordinates if available).

In [2]:
import s3fs
import dask
import xarray as xr
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from hip.analysis.compute.utils import persist_with_progress_bar
from pathlib import Path

In [3]:
country = 'mozambique'  # define country of interest
#directory = Path(f'/s3/scratch/jamie.towner/flood_aa/{country}')  # define main working directory
directory = Path(r"C:\Users\15133\Documents\WFP\flood_hazard\flood_aa\MOZ_training")  # define main working directory

In [4]:
# # Set up the S3 path for the Zarr files
# store = f"s3://wfp-seasmon/input/cds/glofas-reforecast/saf/*/*.zarr"

# # Set up connection to s3 store
# s3 = s3fs.S3FileSystem.current()

# # Define mapper object for multiple files
# remote_files = s3.glob(store)
# store = [s3fs.S3Map(root=f"s3://{file}", s3=s3, check=False) for file in remote_files]

store = list(Path.glob(directory / "data/forecasts/glofas_reforecasts/", "*.zarr"))

# Configure dask to avoid creating large chunks and open the dataset
with dask.config.set(**{"array.slicing.split_large_chunks": True}):
    ds = xr.open_mfdataset(store, decode_coords="all", engine="zarr")

C:\Users\15133\AppData\Local\Temp\ipykernel_12680\3494039067.py:15: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_mfdataset(store, decode_coords="all", engine="zarr")
C:\Users\15133\AppData\Local\Temp\ipykernel_12680\3494039067.py:15: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_mfdataset(store, decode_coords="all", engine="zarr")


In [5]:
# Load the CSV file containing station information (i.e., station name, lat, lon)
# define paths to data
station_info = pd.read_csv(directory / "data/metadata/metadata_observations.csv")

# select only chokwe station for training
station_info = station_info[station_info['station name'] == 'Limpopo_em_Chokwe']

# Define the output directory
out_dir = directory / "data/forecasts/glofas_reforecasts/stations"
Path(out_dir).mkdir(parents=True, exist_ok=True)

In [6]:
# Initialize tqdm progress bar
pbar = tqdm(total=len(station_info), desc="Extracting Data")

# Loop over each station in the metadata file
for index, row in station_info.iterrows():

    point_name = row['station name']
    latitude = row['lisflood_y']
    longitude = row['lisflood_x']
    
    if np.isnan(latitude) or np.isnan(longitude):
        latitude = row['latitude']
        longitude = row['longitude']
    
    # Sanitize the station name for use in a file path (e.g., remove spaces, special characters)
    station_name = "".join(c for c in point_name if c.isalnum() or c in (' ', '_')).replace(' ', '_')
    
    # Define expected output file path
    nc_file_name = os.path.join(out_dir, f"{station_name}.nc")

    # Skip if file already exists
    if os.path.exists(nc_file_name):
        print(f"Skipping {station_name} (already exists)")
        pbar.update(1)
        continue

    # Find the nearest latitude and longitude in the dataset
    lat_index = ds['latitude'].sel(latitude=latitude, method='nearest')
    lon_index = ds['longitude'].sel(longitude=longitude, method='nearest')

    # Extract river discharge data for the nearest point and for all ensemble members
    data_at_point = persist_with_progress_bar(ds['dis24'].sel(latitude=lat_index, longitude=lon_index))

    # Print the actual dimensions for diagnostics
    print("Data at point dimensions:", data_at_point.dims)
    print("Data at point shape:", data_at_point.shape)

    # Extract dates, ensemble member, and step
    dates = ds.time.values
    ensemble_members = ds['dis24'].coords['number'].values
    steps = ds.step.values

    # Create a new xarray dataset for storing data in NetCDF format
    station_ds = xr.Dataset(
        {
            "dis24": (["number", "time", "step"], data_at_point.values)
        },
        coords={
            "number": ensemble_members,
            "time": dates,
            "step": steps
        },
        attrs={
            "description": f"River discharge forecasts for {point_name}",
            "units": "m^3/s"
        }
    )

    # Save the dataset as a NetCDF file
    nc_file_name = os.path.join(out_dir, f"{station_name}.nc")
    station_ds.to_netcdf(nc_file_name)

    # Update the progress bar
    pbar.update(1)

# Close the progress bar
pbar.close()

Extracting Data:   0%|                                                                           | 0/1 [00:00<?, ?it/s]

[########################################] | 100% Completed | 234.67 s
Data at point dimensions: ('number', 'time', 'step')
Data at point shape: (11, 1052, 46)


Extracting Data: 100%|██████████████████████████████████████████████████████████████████| 1/1 [03:59<00:00, 239.55s/it]
